# **DDPM（Denoising Diffusion Probabilistic Models）**

- 是一種 **Generative Model**，這類模型的目標是學會資料的分布，從而可以「生成」出**新的、看起來像是來自訓練資料的 Sample**。

- 在 2020 年提出的，用來解決圖像生成問題，並在品質上逼近甚至超越了 GAN 的效果，但使用的是一種完全不同的方式——**逐步的加噪與去噪（diffusion and denoising）過程**。

| 項目 | 說明 |
|------|------|
| 模型類型 | 生成式模型（Generative Model） |
| 方法核心 | 加噪（forward process） + 去噪（learned reverse process） |
| 訓練目標 | 預測加進去的噪音（MSE loss） |
| 優點 | 穩定訓練、高品質生成 |
| 缺點 | 抽樣速度慢（但 DDIM 有改善） |


### 主要概念 - **Diffusion + Denoising** 並建立在以下兩個核心階段：

1. **前向過程（Forward Process / Diffusion）**
    - 把圖片一步步加入高斯雜訊，模擬退化過程 -> q_sample()
    - 過程可以視為「把一張圖片慢慢加雜訊，直到變成白噪音」

2. **反向過程（Reverse Process / Denoising）**
    - 模型學會一步步「去除雜訊」，從亂數重建圖片 -> p_sample_loop(), p_sample(), p_mean_variance()
    - 讓模型學會如何從噪音一步步還原回原圖
    - 通常用 U-Net 結構的神經網路來預測加入的噪音


### 訓練目標（Loss Function）

- DDPM 的訓練其實非常簡潔。它會從圖片中擷取某一個時間點，加入噪音後，訓練模型去預測當初加進去的噪音：


### **生成過程總體流程圖：**

- Loss fn 的直覺是：**如果模型能準確預測噪音，那它就能正確地「去噪」回圖片**
  
```
訓練期間：
x_0 → q_sample() → x_t（加雜訊）
x_t → 模型 → 預測 epsilon（雜訊）
predicted_noise ≈ 真實 noise → 用來算 loss

生成期間（Inference）：
x_T ~ N(0,I) 或 DIP prior
x_T → p_sample() → x_{T-1} → ... → x_0（逐步還原）
```

- 生成時從純高斯噪音開始，一步步用學到的模型反推回：

```python
x_T = random_noise()
for t in reversed(range(T)):
    x_t-1 = model(x_t, t)  # 預測去噪結果
```


### **延伸發展**：

1. **Improved DDPM**：改進訓練與抽樣方式。
2. **DDIM (Denoising Diffusion Implicit Models)**：可加速抽樣過程。
3. **Latent Diffusion Models (LDM)**：在潛在空間（Latent Space）做 diffusion，加快速度。
4. **Stable Diffusion**：著名的圖像生成模型，就採用了 LDM 結構。


In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
    """Sinusoidal position embeddings for timestep"""
    def __init__(self, dim):
    def forward(self, time):
        '''
       【目的】：將標量的時間步 `t` 轉換成一個固定維度的向量表示，為 U-Net 提供一種方式來區分不同的時間步
       【運作原理】：
            1.【創建頻率】：計算一系列基於對數尺度均勻分佈的不同頻率
            2.【生成 embeddings】：將輸入的時間步與這些頻率相乘，得到不同頻率下的角度
            3.【正弦和餘弦】：對這些角度計算 sin 和 cos
            4.【拼接 torch.cat】：將相同頻率的 sin 和 cos 拼接在一起，形成最終的 time embeddings
            5.【Output】：一個形狀為 `(batch_size, dim)` 的 tensor，表示每個時間步的 embeddings
       【作用】：這種 embedding 方式在 Transformer 模型中被廣泛使用，因為它可以有效地表示序列中元素的位置信息
        '''
        
class SelfAttention(nn.Module):
    """Self-attention block for DDPM"""
    def __init__(self, channels):
    def forward(self, x):
        '''
        【目的】：在特徵圖的空間維度上計算 Self-attention，讓模型捕獲長距離依賴
        `forward(self, x)`：
            【`x`】: 輸入 feature (batch_size, channels, height, width)

       【運作原理】：
            1.【flatten and transpose】：將 `x` 展平成一個 Seq，形狀為`(batch_size, height * width, channels)`
            2.【Multihead-Attention】：輸入的 `x` 同時作為查詢 (query)、鍵 (key) 和值 (value)
            3.【殘差連接 (`+ self.mha(...)`)】：將 Attention 的輸出加回到原始的 `x` 上（殘差連接）
            4.【LayerNor】：對殘差連接後的結果進行 LayerNor
            5.【前饋網路 (`self.ff_self`)】：通過一個包含兩層 Linear 和 GELU 的前饋網路進一步處理特徵
            6.【再次殘差連接 (`+ self.ff_self(...)`) 和層歸一化】：將前饋網路的輸出加回到注意力層的輸出上，再次進行 layerNor
            7.【重塑】：將處理後的序列重新塑造成原始的特徵圖形狀 `(batch_size, channels, height, width)`
            8.【Output】：經過 Self-Attention 處理後的 feature map

       【作用】：允許模型在處理圖像的每個位置時，考慮到所有其他位置的信息，從而更好地理解圖像的全局結構和上下文
        '''

class ResidualBlock(nn.Module):
    """Residual block with optional attention"""
    def __init__(self, in_channels, out_channels, time_emb_dim, dropout, attention=False):
    def forward(self, x, t):
        '''
       【目的】：在 Block 的基礎上增加了殘差連接和 optional attention，使其更適合構建深層網路並處理更複雜的依賴關係
       【運作原理】：
            1.【Conv1】：對輸入 `x` 進行 GroupNorm -> SiLU -> Conv2d
            2.【融入 time embeddings】：將 `t` 通過一個線性層轉換後，加到第一個卷積的輸出上（在空間維度上擴展）
            3.【Conv2】：對融合了時間資訊的 feature map 進行 GroupNorm -> SiLU -> Dropout -> Conv2d
            4.【殘差連接】：如果輸入和輸出 C 不同，則使用一個 1x1 卷積對輸入 `x` 進行通道數調整；否則，直接使用原始輸入
                - 將調整後的輸入加到第二個卷積的輸出上（殘差連接）
            5.【self-attention】：如果 `attention` 為 `True`，則對殘差連接後的結果應用 `SelfAttention`
           6. 【Output】：經過殘差連接和可選注意力處理後的 feature map
       【作用】：`ResidualBlock` 是 U-Net 中的主要特徵處理單元
           - 利用殘差連接促進梯度流動，使用 time embeddings 感知去噪進度，並可選地使用 Self-Attention 來捕獲空間上的長距離依賴
        '''
        

## Model moduels 結構

| 問題 | 方法 |
|------------|----------|
|  U-Net 處理不同時間步的圖像 | `SinusoidalPositionEmbeddings`, `ResidualBlock` |
| 讓 U-Net 可以還原圖像細節 | `ResidualBlock`, `Skip connection` |
| 要模型學會空間結構 | `SelfAttention` |
| 要做圖像 Sampling | Conv2d, ConvTranspose2d |

```
SinusoidalPositionEmbeddings      time → embedding vector
                                  每個時間步代表「圖像被破壞到什麼程度」，模型需要知道
          ↓

                                  時間條件 (x → Conv1 → + time_mlp(t)
ResidualBlock + Self-attention                → Conv2 → + shortcut(x)
                                             → Attention (可選)) +

                                  空間理解 (Feature map → Flatten → Attention → 重建回原形狀)
```

### **在 DDPM 中，U-Net 要學會預測雜訊 ε（epsilon）-> predicted_noise = unet(x_t, t)**

| 重點 | 說明 |
|------|------|
| ➊ U-Net 要做什麼？ | **預測雜訊 epsilon ，幫助 DDPM 去雜訊** |
| ➋ 時間 `t` 怎麼處理？ | 使用 `SinusoidalEmbedding → MLP` 做 time embeddings  |
| ➌ Model Architecture | **Encoder-Decoder + Skip connection** |
| ➍ 接入 DDPM | `self.unet`，Input：`x_t`, `t` Output：noise |
| ➎ Skip connection  | Down-sampling 時儲存，在 Up-sampling 時接回去 concat |


| Module | 作用 | 說明 |
|------|------|---------------|
| `SinusoidalPositionEmbeddings` | 把時間步 `t` 轉成（相似：Transformer 的 position embedding） |  DDPM 每一步都要有「時間條件」 |
| `self.time_mlp` | 用 MLP 處理時間向量 | 每個 ResBlock 都會接收 time embeddings |
| `conv_in` | 把圖片轉成特徵圖 | 圖片進入的第一步 |
| `downs` | 多層 Downsample + ResBlock |  Encoder 部分，提取多尺度特徵 |
| `mid` | 中間 bottleneck |  用兩層 ResBlock 過渡用 |
| `ups` | Decoder + Upsample + Skip connection |將特徵還原回輸出尺寸 |
| `conv_out` | 輸出與輸入圖片一樣大小的 tensor（預測 noise） |  最後輸出預測的雜訊 |


In [ ]:
class UNet(nn.Module):
    """UNet model for DDPM noise prediction"""
    def __init__(self, in_channels, out_channels, hidden_size, time_dim,
                 num_res_blocks, attention_resolutions, dropout):
        '''
        【in_channels、out_channels】：通常兩者 C 相同
        【time_dim (time embedding)】：
            - DDPM 中，時間步 t 會被 embed 成一個向量，作為 U-Net 的額外輸入，
            以告知模型當前處於擴散/去噪過程的哪個階段。

        【hidden_size】：U-Net 中間層數，控制網路的容量
        【num_res_blocks：在每個 Down-Sampeling 和 Up-Sampeling 使用的 ResidualBlock 的數量
            - ResidualBlock 是現代 CNN 中常用的結構，有助於訓練更深的網路

        【attention_resolutions】：一個元組，指定在哪些空間分辨率下使用注意力機制
            - attention 機制可以幫助模型捕獲圖像中的長距離依賴關係
            - 例如：(8, 16) 表示在特徵圖尺寸為 8x8 和 16x16 時使用 attention
       '''

    def forward(self, x, t):
        '''
        【`x`】：輸入的帶噪聲圖像 (`x_t`)。
        【`t`】：當前的時間步（一個批量的時間步）。
        【運作流程】：
            1. 【Time embedding】：將輸入的時間步 `t` 轉換為向量
            2. 【Input】：輸入圖像 `x` 通過 `self.conv_in` 得到初始特徵 `h`
            3. 【Encoder】：`h` 依次通過 `self.downs` 中 -> ResidualBlock -> Down-Sampling
                - 每個 ResidualBlock 都會以 `h` 和 `t` 作為輸入。
                - Encoder 每一層的輸出都會被保存在 `residuals` list 中，用於 Skip-connection
            4. 【Middle】：`h` 通過 `self.mid` 中的 ResidualBlock
            5. 【Decoder】：`h` 依次通過 `self.ups` 中 -> Up-Sampling -> ResidualBlock
                - 在每個 ResidualBlock 之前，`h` 會與 `residuals` list 中對應 Encoder 輸出進行 C 連接
            6. 【Final layer】：最終的 feature map `h` 通過 `self.conv_out` 得到預測的 noise
            7. 【Output】：返回預測的 noise tensor
        '''


### **運作流程：學習一個 Denoising Model ->`unet`，逆轉噪聲添加的過程，實現圖像生成**

1. **前向擴散 (Forward Diffusion):** 通過 `q_sample` 函數，逐步向原始圖像 `x_0` 添加高斯噪聲，經過 $T$ 步後得到一個接近純噪聲的圖像 $x_T$，**這個過程是固定的，不需要學習。**
2. **反向去噪 (Reverse Denoising):** 目標是學習一個反向的過程 $p(x_{t-1} | x_t)$，從 $x_T$ 開始，逐步去除噪聲，最终生成一個新的圖像 $x_0$。反向過程的每一步都依賴於一個去噪模型 `unet`，它預測在當前時間步 `t` 的噪聲。
3. **訓練 (Training):** 通過 `training_losses()`，**模型學習預測在前向擴散過程中添加到圖像中的噪聲**，優化目標是最小化預測噪聲和真實噪聲之間的差異。
4. **採樣 (Sampling):** 訓練完成後，`sample()`從一個隨機噪聲開始，通過 `p_sample_loop` 迭代地應用訓練好的去噪模型，逐步生成新的圖像。

In [ ]:
class DDPM(nn.Module):
    def __init__(self, unet, args):
        '''
        beta_schedule：定義 β 在 T 個時間步上的變化方式
        beta_start, beta_end：noise schedule 中 β 的起始和結束值，β 控制每一步添加到圖像中的噪聲量
        n_steps：擴散過程的總步數 T，noise 會逐步添加到圖像中 T 次。
        '''
    def q_sample(self, x_0, t, noise=None):
        """Forward diffusion process (adding noise to the image)"""
        '''
        運作原理：
        根據 DDPM 的論文，在時間步 t，
        帶噪聲的圖像 x_t 可以通過原始圖像 x_0 和一個與累積的 alphas 相關縮放因子以及一個與 1- alphas 相關的噪聲項直接計算得到
        這個方法就是利用這個公式來快速得到任意時間步的帶噪聲圖像
        '''
    def _extract(self, a, t, shape):
        """Extract coefficients at specified timesteps t and reshape to match input shape"""

    def predict_start_from_noise(self, x_t, t, noise):
        """Predict x_0 from x_t and predicted noise"""

    def q_posterior(self, x_0, x_t, t):
        """Compute parameters for posterior q(x_{t-1} | x_t, x_0)"""
        
    def p_mean_variance(self, x_t, t, clip_denoised=True):
        """Predict mean and variance for reverse process p(x_{t-1} | x_t)"""
        '''
        p_mean_variance(...)：反向去噪過程的核心
            - 使用訓練好的 unet 模型來預測在時間步 t 需要去除的噪聲，
            基於這個預測的噪聲，估計前一個時間步的圖像 x_{t-1} 的 mean 和 variance
        '''
    def p_sample(self, x_t, t, clip_denoised=True):
        """Sample from reverse process p(x_{t-1} | x_t)"""
        '''
        p_sample(...) 運作原理：
            1. 獲取 mean 和 variance：使用 `p_mean_variance` 預測當前時間步 `x_t` 的 denoise mean 和 variance
            2. 採樣 noise：如果當前時間步 `t > 0`，則從標準高斯分佈中採樣一個噪聲張量，在最後一步（`t = 0`），因為要得到最終的生成圖像，通常不添加噪聲
            3. 計算 x_{t-1}：將預測的 mean 加上與預測 variance 和採樣噪聲相關的項，得到去噪後的圖像 `x_{t-1}`
        '''
        
    @torch.no_grad()
    def p_sample_loop(self, shape, device, noise=None, start_step=None, callback=None):
        """Run the entire reverse process to generate samples 純噪聲開始，逐步去除噪聲，最终生成圖像"""
       
    @torch.no_grad()
    def sample(self, batch_size, image_size, channels=3, device=args.device, start_step=None, prior=None):
        """Generate samples using the model"""
        
    def training_losses(self, x_0, t, noise=None):
        """Compute training losses for a single timestep"""
        '''
        DDPM 的標準訓練目標：預測在每個時間步添加到圖像中的噪聲
        運作原理：
            1. 前向擴散：對原始圖像 `x_0` 在隨機時間步 `t` 進行前向擴散，得到帶噪聲的圖像 `x_t` 和實際添加的 `noise`
            2. 噪聲預測：使用 `self.unet(x_t, t)` 預測在時間步 `t` 添加的 noise
            3. 計算 loss：計算預測的噪聲和實際添加的噪聲之間 MSE
        '''
    def forward(self, x, t=None):
        """Forward pass (used for prediction during training)"""
        